<a href="https://colab.research.google.com/github/nhthang04/ECON315-Fall-2025/blob/main/International_Trade_Term_Paper_22_Nov_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build the First Model


## 1. Import all 7 files

In [ ]:
import pandas as pd
import numpy as np
import os

def clean_iso(year):
    # Load file
    df = pd.read_csv(f'/content/ISO{year}.csv')

    # Set the first row as header
    df.columns = df.iloc[0]

    # Drop the first row
    df = df[1:]

    # Remove Land/Sector rows
    df = df[df['Sector number'] != 'Land/Sector']

    # Reset index
    df = df.reset_index(drop=True)

    # Drop row 0 after reset (if it exists, e.g., if there were non-data rows before 'Land/Sector')
    # This line seems redundant if df = df[1:] already handled the first row after setting headers
    # Let's re-evaluate this based on the specific structure of the CSVs if issues arise.
    # For now, let's keep it as is, assuming it handles an additional header-like row after initial processing.
    df = df.drop(index=0).reset_index(drop=True)

    # Rename 'Sector number' to 'Country' and add 'year' column
    df = df.rename(columns={'Sector number': 'exporter'})
    df['year'] = year

    return df


# Load all ISO files from 2018 to 2025
dfs = {}
for year in range(2018, 2025):
    path = f'/content/ISO{year}.csv'
    if os.path.exists(path):
        dfs[year] = clean_iso(year)
        print(f"Loaded ISO{year}.csv — shape: {dfs[year].shape}")
    else:
        print(f"⚠️ File not found: ISO{year}.csv")

# Example: show first 10 rows of 2018
display(dfs[2018].head(10))

Loaded ISO2018.csv — shape: (184, 42)
Loaded ISO2019.csv — shape: (191, 42)
Loaded ISO2020.csv — shape: (190, 42)
Loaded ISO2021.csv — shape: (189, 42)
Loaded ISO2022.csv — shape: (188, 42)
Loaded ISO2023.csv — shape: (187, 42)
Loaded ISO2024.csv — shape: (250, 42)


,exporter,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,NaN,year
0,Afghanistan,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2018
1,Albania,2,NaN,10,6,2,NaN,1,NaN,5,...,10,20,20,33,NaN,2,9,8,13,2018
2,Algeria,NaN,3,22,3,NaN,NaN,3,NaN,NaN,...,6,1,28,7,10,9,4,2,133,2018
3,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2,8,1,1,2,NaN,2,2018
4,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2,8,13,1,NaN,1,NaN,16,2018
5,Argentina,41,62,223,63,12,19,39,1,61,...,79,395,115,590,160,75,181,111,69,2018
6,Armenia,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,2,3,NaN,NaN,NaN,NaN,NaN,NaN,14,2018
7,Australia,9,47,53,3,2,3,6,2,6,...,24,68,350,497,29,7,60,9,2960,2018
8,Austria,2,4,49,5,3,12,8,1,10,...,4,38,40,104,6,19,31,23,120,2018
9,Azerbaijan,NaN,1,6,NaN,NaN,NaN,NaN,1,1,...,NaN,7,9,3,8,7,1,1,59,2018


## 2. Sectors coversion

In [ ]:
import pandas as pd

# Đường dẫn file gốc
file_path = '/content/ISO2018.csv'

# Đọc file nhưng không dùng hàng đầu tiên làm header
df_raw = pd.read_csv(file_path, header=None)

# Hàng thứ 3 trong CSV (index 2) là tên 39 sectors
sectors_row = df_raw.iloc[2]

# Tạo DataFrame mapping Sector number -> Sector name
sectors_df = pd.DataFrame({
    'sector_number': range(1, len(sectors_row)+1),
    'sector_name': sectors_row.values
})

# Nếu muốn xóa NaN (nếu CSV có cột trống)
sectors_df = sectors_df.dropna().reset_index(drop=True)

# Hiển thị
print(sectors_df)

    sector_number                                        sector_name
0               2                  Agriculture, Fishing and Forestry
1               3                               Mining and quarrying
2               4                Food products, beverage and tobacco
3               5                      Textiles and textile products
4               6                       Leather and leather products
5               7              Manufacture of wood and wood products
6               8                     Pulp, paper and paper products
7               9                               Publishing companies
8              10                                 Printing companies
9              11   Manufacture of coke & refined petroleum products
10             12                                       Nuclear fuel
11             13              Chemicals, chemical products & fibres
12             14                                    Pharmaceuticals
13             15                 

In [ ]:
import pandas as pd

# 1. Định nghĩa mapping Aggregated Sector → ISO 39 sectors + HS chapters
aggregated_sectors = [
    {
        "Aggregated_Sector": "Agriculture, Food & Beverages",
        "ISO_39_Sectors": [
            "Agriculture, Fishing and Forestry",
            "Food products, beverage and tobacco"
        ],
        "HS_Chapters": ["01-24"]
    },
    {
        "Aggregated_Sector": "Textiles, Leather & Wood",
        "ISO_39_Sectors": [
            "Textiles and textile products",
            "Leather and leather products",
            "Manufacture of wood and wood products",
            "Pulp, paper and paper products",
            "Publishing companies",
            "Printing companies"
        ],
        "HS_Chapters": ["41-49", "50-67"]
    },
    {
        "Aggregated_Sector": "Chemicals, Rubber & Plastic",
        "ISO_39_Sectors": [
            "Manufacture of coke & refined petroleum products",
            "Nuclear fuel",
            "Chemicals, chemical products & fibres",
            "Pharmaceuticals",
            "Rubber and plastic products"
        ],
        "HS_Chapters": ["27", "28-38", "39-40"]
    },
    {
        "Aggregated_Sector": "Metals & Non-Metallic Minerals",
        "ISO_39_Sectors": [
            "Mining and quarrying",
            "Non-metallic mineral products",
            "Concrete, cement, lime, plaster etc.",
            "Basic metal & fabricated metal products"
        ],
        "HS_Chapters": ["25-26", "68-71", "72-83"]
    },
    {
        "Aggregated_Sector": "Machinery & Electronics",
        "ISO_39_Sectors": [
            "Machinery and equipment",
            "Electrical and optical equipment"
        ],
        "HS_Chapters": ["84", "85", "90-92"]
    },
    {
        "Aggregated_Sector": "Transport Equipment",
        "ISO_39_Sectors": [
            "Shipbuilding",
            "Aerospace",
            "Other transport equipment"
        ],
        "HS_Chapters": ["86-89"]
    }
]

# 2. Chuyển sang DataFrame
rows = []
for ag in aggregated_sectors:
    rows.append({
        "Aggregated_Sector": ag["Aggregated_Sector"],
        "ISO_39_Sectors": ", ".join(ag["ISO_39_Sectors"]),
        "HS_Chapters": ", ".join(ag["HS_Chapters"])
    })

df_aggregated = pd.DataFrame(rows)

# 3. View trên Colab
df_aggregated.head(10)


,Aggregated_Sector,ISO_39_Sectors,HS_Chapters
0,"Agriculture, Food & Beverages","Agriculture, Fishing and Forestry, Food produc...",01-24
1,"Textiles, Leather & Wood","Textiles and textile products, Leather and lea...","41-49, 50-67"
2,"Chemicals, Rubber & Plastic",Manufacture of coke & refined petroleum produc...,"27, 28-38, 39-40"
3,Metals & Non-Metallic Minerals,"Mining and quarrying, Non-metallic mineral pro...","25-26, 68-71, 72-83"
4,Machinery & Electronics,"Machinery and equipment, Electrical and optica...","84, 85, 90-92"
5,Transport Equipment,"Shipbuilding, Aerospace, Other transport equip...",86-89


## 3. Merge and format the data

In [ ]:
# 1. Gộp tất cả dữ liệu wide
df_iso_wide = pd.concat(dfs.values(), ignore_index=True)
display(df_iso_wide.head())

# 2. Reshape wide -> long
df_iso = df_iso_wide.melt(
    id_vars=['exporter', 'year'],
    var_name='sector',
    value_name='number_of_iso'
)

# 3. Đổi tên cột (This line is now redundant as 'Country' is already set)
# df_iso = df_iso.rename(columns={'Country': 'country'})

# 4. Loại NA
df_iso = df_iso.dropna(subset=['number_of_iso']).reset_index(drop=True)

,exporter,1,2,3,4,5,6,7,8,9,...,32.,33.,34.,35.,36.,37.,38.,39.,Column1,Unknown
0,Afghanistan,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Albania,2,NaN,10,6,2,NaN,1,NaN,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Algeria,NaN,3,22,3,NaN,NaN,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 3. Replace missing values with 0
df_iso['number_of_iso'] = df_iso['number_of_iso'].fillna(0)

# 4. Convert number_of_iso to numeric (nếu có ký tự lạ)
df_iso['number_of_iso'] = pd.to_numeric(df_iso['number_of_iso'], errors='coerce').fillna(0)

# 5. Tạo biến logarit ln(ISO + 1)
df_iso['ln_ISO'] = np.log(df_iso['number_of_iso'] + 1)

Note: Xử lý số liệu:Với các ô trống (missing) hoặc không có số liệu, hãy điền là 0. Tạo biến logarit: $ln\_ISO = \ln(ISO\_Count + 1)$.

In [ ]:
display(df_iso.head(20))

,exporter,year,sector,number_of_iso,ln_ISO
0,Albania,2018,1,2,1.098612
1,Argentina,2018,1,41,3.737670
2,Australia,2018,1,9,2.302585
3,Austria,2018,1,2,1.098612
4,Bahrain,2018,1,1,0.693147
5,Bangladesh,2018,1,5,1.791759
6,Belarus,2018,1,8,2.197225
7,Belgium,2018,1,6,1.945910
8,Bosnia and Herzegovina,2018,1,5,1.791759
9,Brazil,2018,1,29,3.401197


In [ ]:
import pandas as pd

# Giả sử df_iso hiện tại
# df_iso = pd.read_csv('df_iso.csv')  # hoặc đã load sẵn

# 1. Định nghĩa mapping Aggregated Sector
aggregated_sectors = [
    {"Aggregated_Sector_ID": "I", "Aggregated_Sector": "Agriculture, Food & Beverages",
     "ISO_39_Sectors": ["Agriculture, Fishing and Forestry","Food products, beverage and tobacco"]},
    {"Aggregated_Sector_ID": "II", "Aggregated_Sector": "Textiles, Leather & Wood",
     "ISO_39_Sectors": ["Textiles and textile products","Leather and leather products","Manufacture of wood and wood products",
                        "Pulp, paper and paper products","Publishing companies","Printing companies"]},
    {"Aggregated_Sector_ID": "III", "Aggregated_Sector": "Chemicals, Rubber & Plastic",
     "ISO_39_Sectors": ["Manufacture of coke & refined petroleum products","Nuclear fuel","Chemicals, chemical products & fibres",
                        "Pharmaceuticals","Rubber and plastic products"]},
    {"Aggregated_Sector_ID": "IV", "Aggregated_Sector": "Metals & Non-Metallic Minerals",
     "ISO_39_Sectors": ["Mining and quarrying","Non-metallic mineral products","Concrete, cement, lime, plaster etc.",
                        "Basic metal & fabricated metal products"]},
    {"Aggregated_Sector_ID": "V", "Aggregated_Sector": "Machinery & Electronics",
     "ISO_39_Sectors": ["Machinery and equipment","Electrical and optical equipment"]},
    {"Aggregated_Sector_ID": "VI", "Aggregated_Sector": "Transport Equipment",
     "ISO_39_Sectors": ["Shipbuilding","Aerospace","Other transport equipment"]}
]

# 2. Tạo mapping sector_name → Aggregated Sector
iso_to_agg = {}
for ag in aggregated_sectors:
    for s in ag["ISO_39_Sectors"]:
        iso_to_agg[s] = {"Aggregated_Sector": ag["Aggregated_Sector"],
                         "Aggregated_Sector_ID": ag["Aggregated_Sector_ID"]}

# --- FIX START ---
# Convert 'sector' column in df_iso to numeric to merge with sectors_df
df_iso['sector'] = pd.to_numeric(df_iso['sector'], errors='coerce')

# Merge df_iso with sectors_df to get 'sector_name'
df_iso = df_iso.merge(sectors_df, left_on='sector', right_on='sector_number', how='left')

# Now, apply mapping using the newly merged 'sector_name' column
df_iso["Aggregated_Sector"] = df_iso["sector_name"].map(lambda x: iso_to_agg.get(x, {}).get("Aggregated_Sector"))
df_iso["Aggregated_Sector_ID"] = df_iso["sector_name"].map(lambda x: iso_to_agg.get(x, {}).get("Aggregated_Sector_ID"))
# --- FIX END ---

# 4. Gộp theo exporter, year, Aggregated_Sector_ID (nếu muốn tổng number_of_iso & ln_ISO)
df_iso_agg = df_iso.groupby(['exporter','year','Aggregated_Sector_ID','Aggregated_Sector'], as_index=False)\
                  .agg({'number_of_iso':'sum','ln_ISO':'sum'})

# 5. View trên Colab
df_iso_agg.head(20)


,exporter,year,Aggregated_Sector_ID,Aggregated_Sector,number_of_iso,ln_ISO
0,Afghanistan,2018,III,"Chemicals, Rubber & Plastic",2,1.386294
1,Afghanistan,2018,IV,Metals & Non-Metallic Minerals,1,0.693147
2,Afghanistan,2019,III,"Chemicals, Rubber & Plastic",2,1.386294
3,Afghanistan,2019,IV,Metals & Non-Metallic Minerals,1,0.693147
4,Afghanistan,2020,III,"Chemicals, Rubber & Plastic",2,1.386294
5,Afghanistan,2020,IV,Metals & Non-Metallic Minerals,2,1.098612
6,Albania,2018,I,"Agriculture, Food & Beverages",6,1.945910
7,Albania,2018,II,"Textiles, Leather & Wood",8,3.583519
8,Albania,2018,III,"Chemicals, Rubber & Plastic",15,5.010635
9,Albania,2018,IV,Metals & Non-Metallic Minerals,35,8.632306


In [ ]:
# 6. Tải file về Colab
df_iso_agg.to_csv('daily_iso_aggregated.csv', index=False)
from google.colab import files
files.download('daily_iso_aggregated.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 4. Synthesize Trade

In [ ]:
import pandas as pd

# Danh sách file CSV và tên muốn đặt cho từng DataFrame (tuỳ chọn)
files = [
    "Cambodia.csv",
    "China1.csv",
    "China2.csv",
    "Indonesia.csv",
    "Lao PDR.csv",
    "Malaysia.csv",
    "Myanmar_Mongolia.csv",
    "Pacific.csv",
    "Philippines.csv",
    "Thailand.csv",
    "Vietnam.csv"
]

# Tạo list DataFrame
dfs = [pd.read_csv(file, encoding="latin1") for file in files]

# Gộp tất cả thành 1 DataFrame
df_all = pd.concat(dfs, ignore_index=True)
df_all = df_all.drop_duplicates(ignore_index=True)

# Kiểm tra kết quả
print(df_all.head())
print(df_all.shape)


  typeCode  freqCode  refPeriodId  refYear  refMonth  period reporterCode  \
0        A  20180101         2018       52      2018     116          KHM   
1        A  20180101         2018       52      2018     116          KHM   
2        A  20180101         2018       52      2018     116          KHM   
3        A  20180101         2018       52      2018     116          KHM   
4        A  20180101         2018       52      2018     116          KHM   

  reporterISO reporterDesc flowCode  ...  netWgt isNetWgtEstimated grossWgt  \
0    Cambodia            X   Export  ...    True               0.0    False   
1    Cambodia            X   Export  ...    True               0.0    False   
2    Cambodia            X   Export  ...    True               0.0    False   
3    Cambodia            X   Export  ...    True               0.0    False   
4    Cambodia            X   Export  ...    True               0.0    False   

   isGrossWgtEstimated    cifvalue    fobvalue primaryValue  \

In [ ]:
df_all.head(100)

,typeCode,freqCode,refPeriodId,refYear,refMonth,period,reporterCode,reporterISO,reporterDesc,flowCode,...,netWgt,isNetWgtEstimated,grossWgt,isGrossWgtEstimated,cifvalue,fobvalue,primaryValue,legacyEstimationFlag,isReported,isAggregate
0,A,20180101,2018,52,2018,116,KHM,Cambodia,X,Export,...,True,0.0,False,NaN,5.121224e+05,5.121224e+05,4,False,True,NaN
1,A,20180101,2018,52,2018,116,KHM,Cambodia,X,Export,...,True,0.0,False,NaN,3.753804e+04,3.753804e+04,4,False,True,NaN
2,A,20180101,2018,52,2018,116,KHM,Cambodia,X,Export,...,True,0.0,False,NaN,7.962692e+05,7.962692e+05,4,False,True,NaN
3,A,20180101,2018,52,2018,116,KHM,Cambodia,X,Export,...,True,0.0,False,NaN,9.691422e+06,9.691422e+06,4,False,True,NaN
4,A,20180101,2018,52,2018,116,KHM,Cambodia,X,Export,...,True,0.0,False,NaN,4.342754e+04,4.342754e+04,4,False,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,A,20180101,2018,52,2018,116,KHM,Cambodia,X,Export,...,True,0.0,False,NaN,6.331768e+04,6.331768e+04,4,False,True,NaN
96,A,20180101,2018,52,2018,116,KHM,Cambodia,X,Export,...,True,0.0,False,NaN,2.944153e+04,2.944153e+04,4,False,True,NaN
97,A,20180101,2018,52,2018,116,KHM,Cambodia,X,Export,...,True,0.0,False,NaN,1.412680e+04,1.412680e+04,4,False,True,NaN
98,A,20180101,2018,52,2018,116,KHM,Cambodia,X,Export,...,True,0.0,False,NaN,8.525026e+05,8.525026e+05,4,False,True,NaN


In [ ]:
df_all1 = df_all[["refPeriodId", "reporterISO", "partnerISO", "isOriginalClassification", "fobvalue"]]
df_all1

,refPeriodId,reporterISO,partnerISO,isOriginalClassification,fobvalue
0,2018,Cambodia,World,1,5.121224e+05
1,2018,Cambodia,World,2,3.753804e+04
2,2018,Cambodia,World,3,7.962692e+05
3,2018,Cambodia,World,4,9.691422e+06
4,2018,Cambodia,World,5,4.342754e+04
...,...,...,...,...,...
455543,2023,Viet Nam,World,64,2.076852e+10
455544,2023,Viet Nam,World,67,8.967313e+07
455545,2023,Viet Nam,World,85,1.327183e+11
455546,2023,Viet Nam,World,88,7.404427e+08


In [ ]:
eap_countries = [
    "Australia", "Brunei Darussalam", "China, Hong Kong SAR", "Japan",
    "Rep. of Korea", "China, Macao SAR", "New Zealand", "Singapore",
    "China", "Malaysia", "Mongolia", "Thailand",
    "Cambodia", "Indonesia", "Lao People's Dem. Rep", "Myanmar", "Papua New Guinea",
    "Philippines", "Timor-Leste", "Viet Nam",
    "Fiji", "Kiribati", "Marshall Isds", "FS Micronesia",
    "Palau", "Samoa", "Solomon Isds", "Tonga", "Tuvalu", "Vanuatu"
]

In [ ]:
# Filter dataframe
df_eap = df_all1[df_all1['partnerISO'].isin(eap_countries)].reset_index(drop=True)

# View
df_eap.head(10)

,refPeriodId,reporterISO,partnerISO,isOriginalClassification,fobvalue
0,2018,Cambodia,Australia,65,51942.56
1,2018,Cambodia,Australia,66,33496.83
2,2018,Cambodia,Australia,70,13.20
3,2018,Cambodia,Australia,73,460674.58
4,2018,Cambodia,Australia,76,40348.60
5,2018,Cambodia,Australia,83,788361.80
6,2018,Cambodia,Australia,84,1213.28
7,2018,Cambodia,Australia,85,574192.75
8,2018,Cambodia,Australia,40,204235.62
9,2018,Cambodia,Australia,39,173516.65


In [ ]:
import pandas as pd

# Giả sử df_eap là dataframe bạn đưa
# df_aggregated là bảng nhóm final
df_aggregated = pd.DataFrame({
    "Aggregated_Sector": [
        "Agriculture, Food & Beverages",
        "Textiles, Leather & Wood",
        "Chemicals, Rubber & Plastic",
        "Metals & Non-Metallic Minerals",
        "Machinery & Electronics",
        "Transport Equipment"
    ],
    "ISO_39_Sectors": [
        "Agriculture, Fishing and Forestry, Food products, beverage and tobacco",
        "Textiles and textile products, Leather and leather products, Manufacture of wood and wood products, Pulp, paper and paper products, Publishing companies, Printing companies",
        "Manufacture of coke & refined petroleum products, Nuclear fuel, Chemicals, chemical products & fibres, Pharmaceuticals, Rubber and plastic products",
        "Mining and quarrying, Non-metallic mineral products, Concrete, cement, lime, plaster etc., Basic metal & fabricated metal products",
        "Machinery and equipment, Electrical and optical equipment",
        "Shipbuilding, Aerospace, Other transport equipment"
    ],
    "HS_Chapters": [
        "01-24",
        "41-49, 50-67",
        "27, 28-38, 39-40",
        "25-26, 68-71, 72-83",
        "84, 85, 90-92",
        "86-89"
    ]
})

# Map Aggregated_Sector sang số La Mã
roman_map = {
    "Agriculture, Food & Beverages": "I",
    "Textiles, Leather & Wood": "II",
    "Chemicals, Rubber & Plastic": "III",
    "Metals & Non-Metallic Minerals": "IV",
    "Machinery & Electronics": "V",
    "Transport Equipment": "VI"
}

# Hàm lấy HS 2-digit
def get_aggregated_sector(hs_code):
    hs2 = int(str(hs_code)[:2])
    for _, row in df_aggregated.iterrows():
        chapters = row['HS_Chapters'].split(',')
        for chap in chapters:
            chap = chap.strip()
            if '-' in chap:
                start, end = map(int, chap.split('-'))
                if start <= hs2 <= end:
                    return row['Aggregated_Sector']
            else:
                if hs2 == int(chap):
                    return row['Aggregated_Sector']
    return None

# Áp dụng map Aggregated_Sector
df_eap['Aggregated_Sector'] = df_eap['isOriginalClassification'].apply(get_aggregated_sector)

# Chuyển sang số La Mã
df_eap['isOriginalClassification_ROMAN'] = df_eap['Aggregated_Sector'].map(roman_map)

# View kết quả
df_eap.head(20)


,refPeriodId,reporterISO,partnerISO,isOriginalClassification,fobvalue,Aggregated_Sector,isOriginalClassification_ROMAN
0,2018,Cambodia,Australia,65,51942.56,"Textiles, Leather & Wood",II
1,2018,Cambodia,Australia,66,33496.83,"Textiles, Leather & Wood",II
2,2018,Cambodia,Australia,70,13.20,Metals & Non-Metallic Minerals,IV
3,2018,Cambodia,Australia,73,460674.58,Metals & Non-Metallic Minerals,IV
4,2018,Cambodia,Australia,76,40348.60,Metals & Non-Metallic Minerals,IV
5,2018,Cambodia,Australia,83,788361.80,Metals & Non-Metallic Minerals,IV
6,2018,Cambodia,Australia,84,1213.28,Machinery & Electronics,V
7,2018,Cambodia,Australia,85,574192.75,Machinery & Electronics,V
8,2018,Cambodia,Australia,40,204235.62,"Chemicals, Rubber & Plastic",III
9,2018,Cambodia,Australia,39,173516.65,"Chemicals, Rubber & Plastic",III


In [ ]:
# merge category
# 1. Bỏ cột Aggregated_Sector
df_eap1 = df_eap.drop(columns=['Aggregated_Sector'])

# 2. Group by theo các cột: refPeriodId, reporterISO, partnerISO, isOriginalClassification_ROMAN
#    và cộng fobvalue
df_eap_grouped = df_eap1.groupby(
    ['refPeriodId', 'reporterISO', 'partnerISO', 'isOriginalClassification_ROMAN'],
    as_index=False
).agg({'fobvalue': 'sum'})

# 3. View kết quả
df_eap_grouped.head(20)

,refPeriodId,reporterISO,partnerISO,isOriginalClassification_ROMAN,fobvalue
0,2018,Cambodia,Australia,I,6.917172e+06
1,2018,Cambodia,Australia,II,9.548260e+07
2,2018,Cambodia,Australia,III,3.982165e+05
3,2018,Cambodia,Australia,IV,1.289780e+06
4,2018,Cambodia,Australia,V,5.832270e+05
5,2018,Cambodia,Australia,VI,1.363225e+06
6,2018,Cambodia,Brunei Darussalam,I,7.313498e+06
7,2018,Cambodia,Brunei Darussalam,II,1.703162e+04
8,2018,Cambodia,Brunei Darussalam,VI,1.000649e+04
9,2018,Cambodia,China,I,1.949948e+08


In [ ]:
import pandas as pd

# 1. Danh sách các Roman I → VI
roman_list = ['I', 'II', 'III', 'IV', 'V', 'VI']

# 2. Danh sách các partner EAP
eap_countries = [
    "Australia", "Brunei Darussalam", "China, Hong Kong SAR", "Japan",
    "Rep. of Korea", "China, Macao SAR", "New Zealand", "Singapore",
    "China", "Malaysia", "Mongolia", "Thailand",
    "Cambodia", "Indonesia", "Lao People's Dem. Rep", "Myanmar", "Papua New Guinea",
    "Philippines", "Timor-Leste", "Viet Nam",
    "Fiji", "Kiribati", "Marshall Isds", "FS Micronesia",
    "Palau", "Samoa", "Solomon Isds", "Tonga", "Tuvalu", "Vanuatu"
]

# 3. Lấy tất cả reporter × refPeriodId có trong df_eap_grouped
unique_reporter_period = df_eap_grouped[['refPeriodId','reporterISO']].drop_duplicates()

# 4. Tạo toàn bộ tổ hợp reporter × partner × Roman × refPeriodId
all_combinations = (
    unique_reporter_period
    .merge(pd.DataFrame({'partnerISO': eap_countries}), how='cross')
    .merge(pd.DataFrame({'isOriginalClassification_ROMAN': roman_list}), how='cross')
)

# 5. Merge với dữ liệu gốc
df_full = all_combinations.merge(
    df_eap_grouped[['refPeriodId','reporterISO','partnerISO','isOriginalClassification_ROMAN','fobvalue']],
    on=['refPeriodId','reporterISO','partnerISO','isOriginalClassification_ROMAN'],
    how='left'
)

# 6. Các fobvalue bị NaN là các hàng chưa có dữ liệu → gán = 0
df_full['fobvalue'] = df_full['fobvalue'].fillna(0)

# 7. Tạo cột trade: 1 nếu fobvalue > 0, else 0
df_full['trade'] = (df_full['fobvalue'] > 0).astype(int)

# 8. Sắp xếp theo reporter, partner, Roman order
roman_order = {r:i for i,r in enumerate(roman_list)}
df_full['roman_order'] = df_full['isOriginalClassification_ROMAN'].map(roman_order)
df_full = df_full.sort_values(by=['refPeriodId','reporterISO','partnerISO','roman_order']).drop(columns='roman_order')

# 9. Xem kết quả
df_full.head(20)

,refPeriodId,reporterISO,partnerISO,isOriginalClassification_ROMAN,fobvalue,trade
0,2018,Cambodia,Australia,I,6.917172e+06,1
1,2018,Cambodia,Australia,II,9.548260e+07,1
2,2018,Cambodia,Australia,III,3.982165e+05,1
3,2018,Cambodia,Australia,IV,1.289780e+06,1
4,2018,Cambodia,Australia,V,5.832270e+05,1
5,2018,Cambodia,Australia,VI,1.363225e+06,1
6,2018,Cambodia,Brunei Darussalam,I,7.313498e+06,1
7,2018,Cambodia,Brunei Darussalam,II,1.703162e+04,1
8,2018,Cambodia,Brunei Darussalam,III,0.000000e+00,0
9,2018,Cambodia,Brunei Darussalam,IV,0.000000e+00,0


In [ ]:
# Xuất file CSV
df_full.to_csv("df_eap_full.csv", index=False)


Add the distance and total trade to form the model

In [ ]:
capital_distances = pd.read_csv("capital_distances.csv")

# -----------------------------
# 2. Chuẩn hóa tên quốc gia
# -----------------------------
def normalize_country(name):
    name = str(name).lower().strip()
    # Chuẩn hóa các viết tắt phổ biến
    replacements = {
        "isds": "islands",
        "rep.": "republic",
        "dpr": "democratic",
        "fs ": "",          # loại bỏ FS, như FS Micronesia → Micronesia
        "hong kong sar": "hong kong",
        "macao sar": "macau",
        ".": "",
        ",": ""
    }
    for k, v in replacements.items():
        name = name.replace(k, v)
    return name

df_full['reporter_norm'] = df_full['reporterISO'].apply(normalize_country)
df_full['partner_norm']  = df_full['partnerISO'].apply(normalize_country)
capital_distances['country_o_norm'] = capital_distances['country_o'].apply(normalize_country)
capital_distances['country_d_norm'] = capital_distances['country_d'].apply(normalize_country)

# -----------------------------
# 3. Hàm fuzzy-like match bằng contain
# -----------------------------
def contains_match(a, b):
    return a in b or b in a

# -----------------------------
# 4. Hàm tìm distance với:
#    exact match ưu tiên
#    A-B = B-A
#    fallback fuzzy contain
# -----------------------------
def find_distance(row, df_dist):
    o = row['reporter_norm']
    d = row['partner_norm']

    # 1. Exact match
    match_exact = df_dist[
        (df_dist['country_o_norm'] == o) & (df_dist['country_d_norm'] == d)
    ]
    if len(match_exact) > 0:
        return match_exact['distance_km'].iloc[0]

    # 2. Reverse exact match (A-B = B-A)
    match_rev_exact = df_dist[
        (df_dist['country_o_norm'] == d) & (df_dist['country_d_norm'] == o)
    ]
    if len(match_rev_exact) > 0:
        return match_rev_exact['distance_km'].iloc[0]

    # 3. Contain / fuzzy match
    match = df_dist[
        df_dist.apply(lambda x: contains_match(o, x['country_o_norm']) and contains_match(d, x['country_d_norm']), axis=1)
    ]
    if len(match) > 0:
        return match['distance_km'].iloc[0]

    # 4. Reverse fuzzy match (A-B = B-A)
    match_rev = df_dist[
        df_dist.apply(lambda x: contains_match(o, x['country_d_norm']) and contains_match(d, x['country_o_norm']), axis=1)
    ]
    if len(match_rev) > 0:
        return match_rev['distance_km'].iloc[0]

    # 5. Không tìm thấy
    return None

# -----------------------------
# 5. Áp dụng cho df_full
# -----------------------------
df_full['distance'] = df_full.apply(lambda row: find_distance(row, capital_distances), axis=1)

# -----------------------------
# 6. Xuất kết quả
# -----------------------------
#print(df_full[['reporterISO','partnerISO','distance']].head(20))

In [ ]:
df_full

,refPeriodId,reporterISO,partnerISO,isOriginalClassification_ROMAN,fobvalue,trade,reporter_norm,partner_norm,distance
0,2018,Cambodia,Australia,I,6.917172e+06,1,cambodia,australia,6980.428298
1,2018,Cambodia,Australia,II,9.548260e+07,1,cambodia,australia,6980.428298
2,2018,Cambodia,Australia,III,3.982165e+05,1,cambodia,australia,6980.428298
3,2018,Cambodia,Australia,IV,1.289780e+06,1,cambodia,australia,6980.428298
4,2018,Cambodia,Australia,V,5.832270e+05,1,cambodia,australia,6980.428298
...,...,...,...,...,...,...,...,...,...
18295,2024,Thailand,Viet Nam,II,1.141919e+09,1,thailand,viet nam,987.636477
18296,2024,Thailand,Viet Nam,III,3.454246e+09,1,thailand,viet nam,987.636477
18297,2024,Thailand,Viet Nam,IV,1.344237e+09,1,thailand,viet nam,987.636477
18298,2024,Thailand,Viet Nam,V,2.745851e+09,1,thailand,viet nam,987.636477


In [ ]:
df_full.head(20)
df_full_distance = df_full[['refPeriodId', 'reporterISO', 'partnerISO', 'isOriginalClassification_ROMAN', 'fobvalue', 'trade', 'distance']]
df_full_distance.to_csv("df_full_with_distance.csv", index=False)
df_full_distance


,refPeriodId,reporterISO,partnerISO,isOriginalClassification_ROMAN,fobvalue,trade,distance
0,2018,Cambodia,Australia,I,6.917172e+06,1,6980.428298
1,2018,Cambodia,Australia,II,9.548260e+07,1,6980.428298
2,2018,Cambodia,Australia,III,3.982165e+05,1,6980.428298
3,2018,Cambodia,Australia,IV,1.289780e+06,1,6980.428298
4,2018,Cambodia,Australia,V,5.832270e+05,1,6980.428298
...,...,...,...,...,...,...,...
18295,2024,Thailand,Viet Nam,II,1.141919e+09,1,987.636477
18296,2024,Thailand,Viet Nam,III,3.454246e+09,1,987.636477
18297,2024,Thailand,Viet Nam,IV,1.344237e+09,1,987.636477
18298,2024,Thailand,Viet Nam,V,2.745851e+09,1,987.636477


In [ ]:
# Merge df_iso_agg vào df_full_distance dựa trên 3 key
df_full_distance = df_full_distance.merge(
    df_iso_agg[['exporter', 'year', 'Aggregated_Sector_ID', 'number_of_iso', 'ln_ISO']],
    left_on=['reporterISO', 'refPeriodId', 'isOriginalClassification_ROMAN'],
    right_on=['exporter', 'year', 'Aggregated_Sector_ID'],
    how='left'
)

# Xóa các cột thừa
df_full_distance = df_full_distance.drop(columns=['exporter', 'year', 'Aggregated_Sector_ID'])

# Kiểm tra kết quả
df_full_distance

,refPeriodId,reporterISO,partnerISO,isOriginalClassification_ROMAN,fobvalue,trade,distance,number_of_iso,ln_ISO
0,2018,Cambodia,Australia,I,6.917172e+06,1,6980.428298,6.0,1.945910
1,2018,Cambodia,Australia,II,9.548260e+07,1,6980.428298,5.0,1.791759
2,2018,Cambodia,Australia,III,3.982165e+05,1,6980.428298,3.0,1.791759
3,2018,Cambodia,Australia,IV,1.289780e+06,1,6980.428298,12.0,4.430817
4,2018,Cambodia,Australia,V,5.832270e+05,1,6980.428298,5.0,1.791759
...,...,...,...,...,...,...,...,...,...
18355,2024,Thailand,Viet Nam,II,1.141919e+09,1,987.636477,27.0,7.454720
18356,2024,Thailand,Viet Nam,III,3.454246e+09,1,987.636477,751.0,18.815658
18357,2024,Thailand,Viet Nam,IV,1.344237e+09,1,987.636477,1079.0,20.641476
18358,2024,Thailand,Viet Nam,V,2.745851e+09,1,987.636477,577.0,8.903543


Add in the import and export GDP

In [ ]:
gdp_1824 = pd.read_csv("GDP_1824.csv", engine='python', skiprows=4)
gdp_1824 = gdp_1824[["Country Name", "Country Code", "2018", "2019", "2020", "2021", "2022", "2023", "2024"]]
gdp_1824

,Country Name,Country Code,2018,2019,2020,2021,2022,2023,2024
0,Aruba,ABW,3.276184e+09,3.395799e+09,2.481857e+09,2.929447e+09,3.279344e+09,3.648573e+09,NaN
1,Africa Eastern and Southern,AFE,1.012291e+12,1.009747e+12,9.334072e+11,1.085605e+12,1.191639e+12,1.133818e+12,1.205974e+12
2,Afghanistan,AFG,1.805322e+10,1.879944e+10,1.995593e+10,1.426000e+10,1.449724e+10,1.715223e+10,NaN
3,Africa Western and Central,AFW,7.778404e+11,8.332889e+11,7.972952e+11,8.581145e+11,8.936399e+11,8.147285e+11,6.700257e+11
4,Angola,AGO,7.945069e+10,7.089796e+10,4.850156e+10,6.650513e+10,1.043997e+11,8.487516e+10,8.039694e+10
...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,7.878760e+09,7.899738e+09,7.717145e+09,9.413404e+09,9.354903e+09,1.046822e+10,1.114860e+10
262,"Yemen, Rep.",YEM,2.160616e+10,NaN,NaN,NaN,NaN,NaN,NaN
263,South Africa,ZAF,4.052607e+11,3.893300e+11,3.379747e+11,4.208869e+11,4.069200e+11,3.806993e+11,4.002607e+11
264,Zambia,ZMB,2.631151e+10,2.330867e+10,1.813776e+10,2.209642e+10,2.916378e+10,2.757796e+10,2.632578e+10


In [ ]:
# -------------------------------
# Full code: Add ln_GDP_ex and ln_GDP_im
# -------------------------------
import pandas as pd
import numpy as np
import math
from difflib import get_close_matches

# -------------------------------
# 1. Load dữ liệu

# Normalize tên nước trong GDP file
gdp_1824['country_norm'] = gdp_1824['Country Name'].str.lower().str.strip()

# -------------------------------
# 2. Remove self-trade
# -------------------------------
df_full_distance = df_full_distance[df_full_distance['reporterISO'].str.lower() != df_full_distance['partnerISO'].str.lower()]

# -------------------------------
# 3. Mapping các trường hợp đặc biệt
# -------------------------------
country_map = {
    'china': 'China',
    'hong kong': 'China, Hong Kong SAR',
    'macau': 'China, Macao SAR',
    'lao people\'s dem. rep.': 'Lao PDR',
    'fs micronesia': 'Micronesia, Fed. Sts.',
    'rep. of korea': 'Korea, Rep.',
    'solomon isds': 'Solomon Islands',
    'marshall isds': 'Marshall Islands',
    'palau': 'Palau',
    'tuvalu': 'Tuvalu',
    'tonga': 'Tonga',
    'samoa': 'Samoa'
}

# -------------------------------
# 4. Hàm lấy GDP theo năm và country
# -------------------------------
def get_gdp(country_name, year):
    country_lower = str(country_name).lower().strip()

    # Kiểm tra mapping trước
    mapped_name = country_map.get(country_lower, None)
    if mapped_name:
        gdp_row = gdp_1824[gdp_1824['Country Name'] == mapped_name]
    else:
        # Nếu không có mapping, tìm tên gần nhất bằng contain hoặc difflib
        match_candidates = gdp_1824['country_norm'].tolist()
        close_match = get_close_matches(country_lower, match_candidates, n=1, cutoff=0.6)
        if len(close_match) > 0:
            gdp_row = gdp_1824[gdp_1824['country_norm'] == close_match[0]]
        else:
            gdp_row = pd.DataFrame()

    if len(gdp_row) == 0:
        return np.nan
    else:
        col = str(year)
        if col in gdp_row.columns:
            return gdp_row[col].values[0]
        else:
            return np.nan

# -------------------------------
# 5. Tạo ln_GDP_ex và ln_GDP_im
# -------------------------------
def safe_ln(x):
    try:
        return math.log(x) if pd.notna(x) and x>0 else np.nan
    except:
        return np.nan

df_full_distance['ln_GDP_ex'] = df_full_distance.apply(lambda row: safe_ln(get_gdp(row['reporterISO'], row['refPeriodId'])), axis=1)
df_full_distance['ln_GDP_im'] = df_full_distance.apply(lambda row: safe_ln(get_gdp(row['partnerISO'], row['refPeriodId'])), axis=1)

# -------------------------------
# 6. Lưu kết quả
# -------------------------------
df_full_distance.to_csv("df_full_distance_with_GDP.csv", index=False)
print("Done! File saved as df_full_distance_with_GDP.csv")

/tmp/ipython-input-3173172948.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full_distance['ln_GDP_ex'] = df_full_distance.apply(lambda row: safe_ln(get_gdp(row['reporterISO'], row['refPeriodId'])), axis=1)


Done! File saved as df_full_distance_with_GDP.csv


/tmp/ipython-input-3173172948.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full_distance['ln_GDP_im'] = df_full_distance.apply(lambda row: safe_ln(get_gdp(row['partnerISO'], row['refPeriodId'])), axis=1)


## 5. Model 1 Running

In [ ]:
#upload file official
df_model1_official = pd.read_csv("df_model1_official.csv")

In [ ]:
# adjust for ln_ISO_x = ln(X + 1)
# 1. Điền 0 vào ô trống
df_model1_official['number_of_iso_x'] = df_model1_official['number_of_iso_x'].fillna(0)

# 2. Tính log(x+1)
df_model1_official['ln_ISO_x'] = np.log(df_model1_official['number_of_iso_x'] + 1)
df_model1_official

# drop ln_GDP_im = 0
df_model1_official = df_model1_official[
    df_model1_official['ln_GDP_im'].notna()
]

# Save
df_model1_official.to_csv("df_model1_cleaned.csv", index=False)

print("Done! Saved as df_model1_cleaned.csv")

Done! Saved as df_model1_cleaned.csv


1. Đây là mô hình để kiểm tra giả thuyết "Vé vào cửa" (Ticket-to-Entry).

2. Câu hỏi: Liệu có nhiều chứng chỉ ISO hơn thì xác suất để một ngành hàng xuất khẩu được sang nước khác có tăng lên không?

3. Phương trình:

$$Pr(Trade = 1) = \Phi(\alpha + \beta_1 \cdot \ln(ISO) + \beta_2 \cdot \ln(Distance) + \beta_3 \cdot \ln(GDP_{Ex}) + \beta_4 \cdot \ln(GDP_{Im}) + \text{Fixed Effects})$$

4. Kỳ vọng: Hệ số $\beta_1$ (của ln_ISO) sẽ mang dấu Dương (+) và có ý nghĩa thống kê (P-value < 0.05).

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats # Import scipy.stats for norm distribution functions

# 1. Đọc dữ liệu
df_model1_cleaned = pd.read_csv('df_model1_cleaned.csv')

# 2. Chuẩn bị biến số
# Handle potential zero distances before taking log to avoid -inf
# Replace 0 with a very small positive number (e.g., 1e-9)
df_model1_cleaned['distance_for_log'] = df_model1_cleaned['distance'].replace(0, 1e-9)
df_model1_cleaned['ln_dist'] = np.log(df_model1_cleaned['distance_for_log'])

# Chuyển các biến phân loại sang dạng category
df_model1_cleaned['Year_Cat'] = df_model1_cleaned['refPeriodId'].astype('category')
df_model1_cleaned['Sector_Cat'] = df_model1_cleaned['isOriginalClassification_ROMAN'].astype('category')

# Ensure no NaNs or Infs are present in the final dataset for model fitting
# Although previous steps tried to clean, a final check before model fit is good practice.
df_model1_cleaned.replace([np.inf, -np.inf], np.nan, inplace=True)
df_model1_cleaned.dropna(
    subset=['ln_ISO_x', 'ln_dist', 'ln_GDP_ex', 'ln_GDP_im', 'trade', 'Sector_Cat', 'Year_Cat'],
    inplace=True
)

# 3. Xây dựng phương trình
formula = "trade ~ ln_ISO_x + ln_dist + ln_GDP_ex + ln_GDP_im + C(Sector_Cat) + C(Year_Cat)"

# 4. Chạy mô hình Probit
print("Probit model running... Please wait")
probit_model = smf.probit(formula, data=df_model1_cleaned).fit(disp=0)

# 5. Kết quả
print(probit_model.summary())

# 6. Tính Inverse Mills Ratio (IMR)
pred_z = probit_model.predict(linear=True)  # z-score
pdf = stats.norm.pdf(pred_z) # Corrected to use scipy.stats.norm
cdf = stats.norm.cdf(pred_z) # Corrected to use scipy.stats.norm
df_model1_cleaned['imr'] = pdf / cdf

# 7. Lưu file cho giai đoạn 2
df_model1_cleaned.to_csv('df_model2_draft.csv', index=False)
print("\nDa tinh xong IMR va luu file 'df_model2_ready.csv' cho Giai doan 2.")

Probit model running... Please wait
                          Probit Regression Results                           
Dep. Variable:                  trade   No. Observations:                17622
Model:                         Probit   Df Residuals:                    17606
Method:                           MLE   Df Model:                           15
Date:                Sat, 22 Nov 2025   Pseudo R-squ.:                  0.3263
Time:                        20:43:09   Log-Likelihood:                -8018.0
converged:                       True   LL-Null:                       -11902.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -8.5708      0.239    -35.912      0.000      -9.039      -8.103
C(Sector_Cat)[T.II]     -0.1593      0.039     -4.039      0.000 

/usr/local/lib/python3.12/dist-packages/statsmodels/discrete/discrete_model.py:530: FutureWarning: linear keyword is deprecated, use which="linear"
  warnings.warn(msg, FutureWarning)



Da tinh xong IMR va luu file 'df_model2_ready.csv' cho Giai doan 2.


# Build the second model

## Consolidate variables

In [ ]:
import pandas as pd

# Danh sách file Model2
files = [
    "Vietnam21to30_Model2.csv",
    "Vietnam11to20_Model2.csv",
    "Vietnam1to10_Model2.csv",
    "TimorTonga_Model2.csv",
    "Thailand24_Model2.csv",
    "Thailand23_Model2.csv",
    "Thailand22_Model2.csv",
    "Thailand21_Model2.csv",
    "Thailand20_Model2.csv",
    "Thailand19_Model2.csv",
    "Thailand18_Model2.csv",
    "SamoaSolo_Model2.csv",
    "Philippines2324_Model2.csv",
    "Philippines1822_Model2.csv",
    "PalauPapua_Model2.csv",
    "Myanmar_Model2.csv",
    "Mongolia_Model2.csv",
    "Malaysia21to30_Model2.csv",
    "Malaysia11to20_Model2.csv",
    "Malaysia1to10_Model2.csv",
    "Laos_Model2.csv",
    "Kiribati_Model2.csv",
    "Indonesia2223_Model2.csv",
    "Indonesia2021_Model2.csv",
    "Indonesia1819_Model2.csv",
    "Indonesia24_Model2.csv",
    # "Vietnam21to30_Model2.csv",  # <- duplicate, loại bỏ
    "ChinaFirst10_Model2.csv",
    "China21to30_Model2.csv",
    "China11to20_Model2.csv",
    "Cambodia_Model2.csv"
]

# -----------------------------------------
# 1) Hàm đọc file CSV KHÔNG BAO GIỜ LỆCH CỘT
# -----------------------------------------
def safe_read_csv(file):
    print(f"\nĐang xử lý file: {file}")

    # Bước 1: xác định số cột tối đa dựa vào dấu phẩy
    with open(file, "r", encoding="cp1252", errors="ignore") as f:
        lines = f.readlines()

    col_counts = [line.count(",") + 1 for line in lines]
    max_cols = max(col_counts)

    print(f"   → Phát hiện {max_cols} cột tối đa trong file")

    # Bước 2: tạo danh sách tên cột tạm
    colnames = [f"col_{i}" for i in range(max_cols)]

    # Bước 3: đọc file theo số cột cố định
    df = pd.read_csv(
        file,
        header=None,
        names=colnames,
        encoding="cp1252",
        on_bad_lines="skip"   # vẫn đọc được mọi dòng mà không lệch
    )

    print(f"   → Đọc xong file, shape = {df.shape}")
    return df


# -----------------------------------------
# 2) Đọc từng file bằng safe_read_csv
# -----------------------------------------
dfs = []
for f in files:
    df = safe_read_csv(f)
    dfs.append(df)

# -----------------------------------------
# 3) Merge toàn bộ file
# -----------------------------------------
df_all_model2 = pd.concat(dfs, ignore_index=True)

print("\n===============================")
print("MERGE DONE!")
print("Tổng số dòng:", len(df_all_model2))
print("Tổng số cột:", df_all_model2.shape[1])
print("===============================")



Đang xử lý file: Vietnam21to30_Model2.csv
   → Phát hiện 66 cột tối đa trong file


/tmp/ipython-input-1545974742.py:57: DtypeWarning: Columns (2,3,4,5,6,11,14,19,20,22,23,26,27,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
/tmp/ipython-input-1545974742.py:57: DtypeWarning: Columns (2,3,4,5,6,11,14,19,20,22,23,26,27,29,31,32,33,35,36,37,38,39,40,41,42,43,44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


   → Đọc xong file, shape = (17098, 66)

Đang xử lý file: Vietnam11to20_Model2.csv
   → Phát hiện 66 cột tối đa trong file
   → Đọc xong file, shape = (13394, 66)

Đang xử lý file: Vietnam1to10_Model2.csv
   → Phát hiện 67 cột tối đa trong file


/tmp/ipython-input-1545974742.py:57: DtypeWarning: Columns (2,3,4,5,6,11,14,19,20,22,23,26,27,29,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
/tmp/ipython-input-1545974742.py:57: DtypeWarning: Columns (2,3,4,5,6,11,14,19,20,22,23,26,27,29,31,32,33,35,36,37,38,39,40,41,42,43,44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


   → Đọc xong file, shape = (26914, 67)

Đang xử lý file: TimorTonga_Model2.csv
   → Phát hiện 66 cột tối đa trong file
   → Đọc xong file, shape = (8330, 66)

Đang xử lý file: Thailand24_Model2.csv
   → Phát hiện 67 cột tối đa trong file


...

   → Đọc xong file, shape = (40244, 66)

Đang xử lý file: China11to20_Model2.csv
   → Phát hiện 66 cột tối đa trong file


/tmp/ipython-input-1545974742.py:57: DtypeWarning: Columns (2,3,4,5,6,11,14,19,20,22,23,26,27,29,31,32,33,35,36,37,38,39,40,41,42,43,44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


   → Đọc xong file, shape = (56757, 66)

Đang xử lý file: Cambodia_Model2.csv
   → Phát hiện 66 cột tối đa trong file


/tmp/ipython-input-1545974742.py:57: DtypeWarning: Columns (2,3,4,5,6,11,14,19,20,22,23,26,27,29,31,32,33,35,36,37,38,39,40,41,42,43,44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


   → Đọc xong file, shape = (42731, 66)

MERGE DONE!
Tổng số dòng: 1330393
Tổng số cột: 67


In [ ]:
df_all_model2

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_57,col_58,col_59,col_60,col_61,col_62,col_63,col_64,col_65,col_66
0,typeCode,freqCode,refPeriodId,refYear,refMonth,period,reporterCode,reporterISO,reporterDesc,flowCode,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C,A,20180101,2018,52,2018,704,VNM,Viet Nam,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C,A,20180101,2018,52,2018,704,VNM,Viet Nam,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C,A,20180101,2018,52,2018,704,VNM,Viet Nam,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C,A,20180101,2018,52,2018,704,VNM,Viet Nam,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1330388,C,A,20240101,2024,52,2024,116,KHM,Cambodia,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1330389,C,A,20240101,2024,52,2024,116,KHM,Cambodia,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1330390,C,A,20240101,2024,52,2024,116,KHM,Cambodia,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1330391,C,A,20240101,2024,52,2024,116,KHM,Cambodia,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd

# Giả sử df_all_model2 là DataFrame hiện tại
df_all_model2.columns = df_all_model2.iloc[0]  # dùng dòng đầu tiên làm header
df_all_model21 = df_all_model2[1:].reset_index(drop=True)  # bỏ dòng đầu tiên cũ và reset index
df_all_model21["cmdCode"]

,cmdCode
0,8901
1,8412
2,8704
3,8711
4,9102
...,...
1330387,6004
1330388,6404
1330389,6702
1330390,9404


In [ ]:
df_model1_cleaned

,refPeriodId,reporterISO,partnerISO,isOriginalClassification_ROMAN,fobvalue,trade,country_1,country_2,distance,number_of_iso_x,ln_ISO_x,ln_GDP_ex,ln_GDP_im,distance_for_log,ln_dist,Year_Cat,Sector_Cat,imr
0,2018,Cambodia,Australia,I,6.917172e+06,1,Australia,Cambodia,6980.428298,6.0,1.945910,24.224185,27.987162,6980.428298,8.850866,2018,I,0.212985
1,2018,Cambodia,Australia,II,9.548260e+07,1,Australia,Cambodia,6980.428298,5.0,1.791759,24.224185,27.987162,6980.428298,8.850866,2018,II,0.270997
2,2018,Cambodia,Australia,III,3.982165e+05,1,Australia,Cambodia,6980.428298,3.0,1.386294,24.224185,27.987162,6980.428298,8.850866,2018,III,0.337141
3,2018,Cambodia,Australia,IV,1.289780e+06,1,Australia,Cambodia,6980.428298,12.0,2.564949,24.224185,27.987162,6980.428298,8.850866,2018,IV,0.331242
4,2018,Cambodia,Australia,V,5.832270e+05,1,Australia,Cambodia,6980.428298,5.0,1.791759,24.224185,27.987162,6980.428298,8.850866,2018,V,0.302842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17617,2024,Thailand,Viet Nam,II,1.141919e+09,1,Thailand,Viet Nam,987.636477,27.0,3.332205,26.989349,26.889499,987.636477,6.895315,2024,II,0.066983
17618,2024,Thailand,Viet Nam,III,3.454246e+09,1,Thailand,Viet Nam,987.636477,751.0,6.622736,26.989349,26.889499,987.636477,6.895315,2024,III,0.043453
17619,2024,Thailand,Viet Nam,IV,1.344237e+09,1,Thailand,Viet Nam,987.636477,1079.0,6.984716,26.989349,26.889499,987.636477,6.895315,2024,IV,0.050428
17620,2024,Thailand,Viet Nam,V,2.745851e+09,1,Thailand,Viet Nam,987.636477,577.0,6.359574,26.989349,26.889499,987.636477,6.895315,2024,V,0.041830


In [ ]:
# category conversion
df_all_model21["hs2"] = df_all_model21["cmdCode"].astype(str).str[:2]
df_all_model21["hs2"] = pd.to_numeric(df_all_model21["hs2"], errors="coerce")

df_all_model21["hs2"]

,hs2
0,89.0
1,84.0
2,87.0
3,87.0
4,91.0
...,...
1330387,60.0
1330388,64.0
1330389,67.0
1330390,94.0


In [ ]:

def map_category2(hs):
    if 1 <= hs <= 24:
        return "I"
    if (41 <= hs <= 49) or (50 <= hs <= 67):
        return "II"
    if hs == 27 or (28 <= hs <= 38) or (39 <= hs <= 40):
        return "III"
    if (25 <= hs <= 26) or (68 <= hs <= 71) or (72 <= hs <= 83):
        return "IV"
    if (84 <= hs <= 85) or (90 <= hs <= 92):
        return "V"
    if 86 <= hs <= 89:
        return "VI"
    return None

df_all_model21["Category"] = df_all_model21["hs2"].apply(map_category2)
df_all_model21


,typeCode,freqCode,refPeriodId,refYear,refMonth,period,reporterCode,reporterISO,reporterDesc,flowCode,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hs2,Category
0,C,A,20180101,2018,52,2018,704,VNM,Viet Nam,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.0,VI
1,C,A,20180101,2018,52,2018,704,VNM,Viet Nam,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84.0,V
2,C,A,20180101,2018,52,2018,704,VNM,Viet Nam,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87.0,VI
3,C,A,20180101,2018,52,2018,704,VNM,Viet Nam,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87.0,VI
4,C,A,20180101,2018,52,2018,704,VNM,Viet Nam,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.0,V
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1330387,C,A,20240101,2024,52,2024,116,KHM,Cambodia,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,II
1330388,C,A,20240101,2024,52,2024,116,KHM,Cambodia,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0,II
1330389,C,A,20240101,2024,52,2024,116,KHM,Cambodia,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.0,II
1330390,C,A,20240101,2024,52,2024,116,KHM,Cambodia,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.0,None


In [ ]:
# 1. Chắc chắn bỏ các dòng header lặp lại khi merge
df_all_model21 = df_all_model21[df_all_model21['refYear'] != 'refYear']  # remove repeated header rows

# 2. Chọn cột
df_trim21 = df_all_model21[['refYear', 'reporterDesc', 'partnerDesc', 'Category', 'netWgt']]

# 3. Đổi type netWgt sang numeric
df_trim21['netWgt'] = pd.to_numeric(df_trim21['netWgt'], errors='coerce')

# 4. Fill NaN nếu muốn (tránh nhảy dòng khi group)
df_trim21['netWgt'] = df_trim21['netWgt'].fillna(0)

# Bây giờ df_trim2 sẽ giữ nguyên thứ tự dòng gốc
df_trim21

/tmp/ipython-input-436071681.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trim21['netWgt'] = pd.to_numeric(df_trim21['netWgt'], errors='coerce')
/tmp/ipython-input-436071681.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trim21['netWgt'] = df_trim21['netWgt'].fillna(0)


,refYear,reporterDesc,partnerDesc,Category,netWgt
0,2018,Viet Nam,Rep. of Korea,VI,0.00
1,2018,Viet Nam,Philippines,V,0.00
2,2018,Viet Nam,Philippines,VI,0.00
3,2018,Viet Nam,Philippines,VI,0.00
4,2018,Viet Nam,Philippines,V,0.00
...,...,...,...,...,...
1330387,2024,Cambodia,Bangladesh,II,264108.00
1330388,2024,Cambodia,Bangladesh,II,403.20
1330389,2024,Cambodia,Bangladesh,II,1.22
1330390,2024,Cambodia,Bangladesh,None,1.87


In [ ]:
import pandas as pd

# Giả sử df_trim2 là DataFrame hiện tại
# 1. Nếu có NaN trong netWgt, thay bằng 0
df_trim21['netWgt'] = df_trim21['netWgt'].fillna(0)

# 2. Group by 4 cột đầu, cộng netWgt
df_grouped21 = df_trim21.groupby(
    ['refYear', 'reporterDesc', 'partnerDesc', 'Category'],
    as_index=False
)['netWgt'].sum()

# 3. Xem kết quả
df_grouped21
df_grouped21.to_csv('df_grouped21.csv', index=False)



/tmp/ipython-input-2136957169.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trim21['netWgt'] = df_trim21['netWgt'].fillna(0)


In [ ]:
import pandas as pd

# -------------------------
# 1) Chuẩn hóa tên nước
# -------------------------
def normalize_name(name):
    return str(name).strip().lower()

df_model1_cleaned["reporterISO_norm"] = df_model1_cleaned["reporterISO"].apply(normalize_name)
df_model1_cleaned["partnerISO_norm"] = df_model1_cleaned["partnerISO"].apply(normalize_name)
df_grouped21["reporterDesc_norm"] = df_grouped21["reporterDesc"].apply(normalize_name)
df_grouped21["partnerDesc_norm"] = df_grouped21["partnerDesc"].apply(normalize_name)

# -------------------------
# 2) Chuẩn hóa các case đặc biệt
# -------------------------
special_mapping = {
    "lao dpr": "lao",
    "lao pdr": "lao",
    "viet nam": "viet nam",
    "us": "united states",
    "america": "united states",
}
for col in ["reporterISO_norm","partnerISO_norm"]:
    df_model1_cleaned[col] = df_model1_cleaned[col].replace(special_mapping)
for col in ["reporterDesc_norm","partnerDesc_norm"]:
    df_grouped21[col] = df_grouped21[col].replace(special_mapping)

# -------------------------
# 3) Group df_grouped21 để sum netWgt
# -------------------------
df_grouped_sum = df_grouped21.groupby(
    ["refYear","reporterDesc_norm","partnerDesc_norm","Category"], as_index=False
)["netWgt"].sum()

# -------------------------
# 4) Merge gần đúng (cross join nhỏ) để vectorized contains
# -------------------------
# Chỉ merge theo refYear & Category trước
df_model1_cleaned["refPeriodId"] = df_model1_cleaned["refPeriodId"].astype(int)
df_grouped_sum["refYear"] = df_grouped_sum["refYear"].astype(int)

df_merged = df_model1_cleaned.merge(
    df_grouped_sum,
    left_on=["refPeriodId","isOriginalClassification_ROMAN"],
    right_on=["refYear","Category"],
    how="left",
    suffixes=("","_grp")
)

# Vectorized contain check
mask_reporter = df_merged["reporterDesc_norm"].str.contains(df_merged["reporterISO_norm"], na=False)
mask_partner = df_merged["partnerDesc_norm"].str.contains(df_merged["partnerISO_norm"], na=False)
df_merged["num_weight"] = df_merged["netWgt"].where(mask_reporter & mask_partner, 0)

# -------------------------
# 5) Gán lại vào df_model1_cleaned
# -------------------------
df_model1_cleaned["num_weight"] = df_merged["num_weight"]

# -------------------------
# 6) Kiểm tra
# -------------------------
print(df_model1_cleaned[["refPeriodId","reporterISO","partnerISO","isOriginalClassification_ROMAN","num_weight"]].head())
